# library

In [1]:
import sys
import os
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from sklearn.model_selection import train_test_split
from imutils import paths
import cv2

/Users/evan/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/evan/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/evan/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/evan/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/f

Instructions for updating:
non-resource variables are not supported in the long term


# character table

In [2]:
number = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
alphbet = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T',
                 'U', 'V', 'W', 'X', 'Y', 'Z']
chinese = ['zh_cuan', 'zh_e', 'zh_gan', 'zh_gan1', 'zh_gui', 'zh_gui1', 'zh_hei', 'zh_hu', 'zh_ji', 'zh_jin',
                'zh_jing', 'zh_jl', 'zh_liao', 'zh_lu', 'zh_meng', 'zh_min', 'zh_ning', 'zh_qing', 'zh_qiong',
                'zh_shan', 'zh_su', 'zh_sx', 'zh_wan', 'zh_xiang', 'zh_xin', 'zh_yu', 'zh_yu1', 'zh_yue', 'zh_yun',
                'zh_zang', 'zh_zhe']

# Functions (made in a same class)

In [3]:
class char_recognition_cnn_net:
    
    # initial some useful variables
    def __init__(self):
        self.dataset = number + alphbet + chinese
        self.dataset_length = len(self.dataset)
        self.y_len = len(self.dataset)
        self.batch_size = 100
        self.image_size = 20
        self.x_place = tf.placeholder(dtype=tf.float32, shape=[None, self.image_size, self.image_size], name='x_place')
        self.y_place = tf.placeholder(dtype=tf.float32, shape=[None, self.y_len], name='y_place')
        self.keep_place = tf.placeholder(dtype=tf.float32, name='keep_place')
        
    #The cnn architecture we used for this model training
    def cnn_architecture(self):
        #input layer - binary input only 1 channel
        x_inputs = tf.reshape(self.x_place, shape=[-1, 20, 20, 1])
        
        #convelutional layer 1
        conv_weight1 = tf.Variable(tf.random_normal(shape=[3, 3, 1, 32], stddev=0.01), dtype=tf.float32)
        conv_bias1 = tf.Variable(tf.random_normal(shape=[32]), dtype=tf.float32)
        conv_layer1 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(x_inputs,filter=conv_weight1,strides=[1,1,1,1],padding='SAME'),conv_bias1))
        conv_layer1 = tf.nn.max_pool(conv_layer1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        conv_layer1 = tf.nn.dropout(conv_layer1, self.keep_place)
        
        #convelutional layer 2
        conv_weight2 = tf.Variable(tf.random_normal(shape=[3, 3, 32, 64], stddev=0.01), dtype=tf.float32)
        conv_bias2 = tf.Variable(tf.random_normal(shape=[64]), dtype=tf.float32)
        conv_layer2 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(conv_layer1,filter=conv_weight2,strides=[1,1,1,1],padding='SAME'),conv_bias2))
        conv_layer2 = tf.nn.max_pool(conv_layer2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        conv_layer2 = tf.nn.dropout(conv_layer2, self.keep_place)
        
        #convelutional layer 3
        conv_weight3 = tf.Variable(tf.random_normal(shape=[3, 3, 64, 128], stddev=0.01), dtype=tf.float32)
        conv_bias3 = tf.Variable(tf.random_normal(shape=[128]), dtype=tf.float32)
        conv_layer3 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(conv_layer2,filter=conv_weight3,strides=[1,1,1,1],padding='SAME'),conv_bias3))
        conv_layer3 = tf.nn.max_pool(conv_layer3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        conv_layer3 = tf.nn.dropout(conv_layer3, self.keep_place)
        
        #full convelutional layer
        conv_layer_out = tf.reshape(conv_layer3, shape=[-1, 3 * 3 * 128])
        
        #fully connected layer 1
        fully_con_weight1 = tf.Variable(tf.random_normal(shape=[3 * 3 * 128, 1024], stddev=0.01), dtype=tf.float32)
        fully_con_bias1 = tf.Variable(tf.random_normal(shape=[1024]), dtype=tf.float32)
        fully_con_layer1 = tf.nn.relu(tf.add(tf.matmul(conv_layer_out, fully_con_weight1), fully_con_bias1))
        fully_con_layer1 = tf.nn.dropout(fully_con_layer1, self.keep_place)
        
        
        #fully connected layer 2
        fully_con_weight2 = tf.Variable(tf.random_normal(shape=[1024, 1024], stddev=0.01), dtype=tf.float32)
        fully_con_bias2 = tf.Variable(tf.random_normal(shape=[1024]), dtype=tf.float32)
        fully_con_layer2 = tf.nn.relu(tf.add(tf.matmul(fully_con_layer1, fully_con_weight2), fully_con_bias2))
        fully_con_layer2 = tf.nn.dropout(fully_con_layer2, self.keep_place)

        
        #fully connected layer 3, output layer
        fully_con_weight3 = tf.Variable(tf.random_normal(shape=[1024, self.dataset_length], stddev=0.01), dtype=tf.float32)
        fully_con_bias3 = tf.Variable(tf.random_normal(shape=[self.dataset_length]), dtype=tf.float32)
        fully_con_layer3 = tf.add(tf.matmul(fully_con_layer2, fully_con_weight3), fully_con_bias3, name='out_put')
        
        #output the last layer, output shape is class numbers
        return fully_con_layer3
    
    
    #initial data and labels x,y as two np.array
    def init_data(self,dir):
        X = []
        y = []
        #check the path exists or not
        if not os.path.exists(data_dir):
            raise ValueError('No such file folder')
        files = list(paths.list_images(dir))
        for file in files:
            src_img = cv2.imread(file, cv2.COLOR_BGR2GRAY)
            if src_img.ndim == 3:
                continue
            #resize the image as target size
            resize_img = cv2.resize(src_img, (20, 20))
            X.append(resize_img)
            # get the file folder name for each class
            dir_name = file.split(os.path.sep)[-2]
            vector_y = [0 for i in range(len(self.dataset))]
            index_y = self.dataset.index(dir_name)
            vector_y[index_y] = 1
            y.append(vector_y)
        X = np.array(X)
        y = np.array(y).reshape(-1, self.dataset_length)
        return X, y


    
    def train(self,data_dir,save_model_path):
        #process and read x, y data and labels use the function
        X, y = self.init_data(data_dir)
        print('loaded' + str(len(y)) + 'dataset')
        
        #split data as train and validation data set for 8 to 2 ratio
        train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=0)
        #read the architecture
        out_puts = self.cnn_architecture()
        predict = tf.nn.softmax(out_puts)
        predict = tf.argmax(predict, axis=1)
        actual_y_value = tf.argmax(self.y_place, axis=1)
        accuracy = tf.reduce_mean(tf.cast(tf.equal(predict, actual_y_value), dtype=tf.float32))
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=out_puts, labels=self.y_place))
        #optimizer and learning rate
        opt = tf.train.AdamOptimizer(learning_rate=0.001)
        train_step = opt.minimize(cost)
        
        #start the train session
        sess = tf.Session()
        init = tf.global_variables_initializer()
        sess.run(init)
        step = 0
        savers = tf.train.Saver()
        while True:
            train_index = np.random.choice(len(train_x), self.batch_size, replace=False)
            train_randx = train_x[train_index]
            train_randy = train_y[train_index]
            _, loss = sess.run([train_step, cost],
                               feed_dict={self.x_place:train_randx,self.y_place:train_randy,self.keep_place:0.75})
            step += 1

            if step % 10 == 0:
                test_index = np.random.choice(len(test_x), self.batch_size, replace=False)
                test_randx = test_x[test_index]
                test_randy = test_y[test_index]
                acc = sess.run(accuracy,feed_dict={self.x_place : test_randx, self.y_place : test_randy,
                                                   self.keep_place : 1.0})
                print(step, loss)
                # stop train both step >500 and accuracy > 99%
                if step % 50 == 0:
                    print('accuracy:' + str(acc))
                if step % 500 == 0:
                    savers.save(sess, save_model_path, global_step=step)
                if acc > 0.99 and step > 500:
                    #save model
                    savers.save(sess, save_model_path, global_step=step)
                    #close the train session
                    sess.close()
                    break
                    
                    
    #initial data x as one np.array
    def init_testData(self,dir):
        test_X = []
        #check the path exists or not
        if not os.path.exists(test_dir):
            raise ValueError('No such file folder')
        files = list(paths.list_images(dir))
        for file in files:
            src_img = cv2.imread(file, cv2.COLOR_BGR2GRAY)
            if src_img.ndim == 3:
                continue
            resize_img = cv2.resize(src_img, (20, 20))
            test_X.append(resize_img)
        test_X = np.array(test_X)
        return test_X
    

    def test(self,x_images,model_path):
        text_list = []
        out_puts = self.cnn_architecture()
        predict = tf.nn.softmax(out_puts)
        predict = tf.argmax(predict, axis=1)
        savers = tf.train.Saver()
        #start session
        sess = tf.Session()
        sess.run(tf.global_variables_initializer())
        savers.restore(sess, model_path)
        preds = sess.run(predict, feed_dict={self.x_place: x_images, self.keep_place: 1.0})
        for i in range(len(preds)):
            pred = preds[i].astype(int)
            text_list.append(self.dataset[pred])
        #close session
        sess.close()
        return text_list
    


# Main Program

In [4]:
cur_pos = sys.path[0]
data_dir = os.path.join(cur_pos, 'Dataset/char_train/')
test_dir = os.path.join(cur_pos, 'Dataset/char_test')
train_model_path = os.path.join(cur_pos, './Dataset/models/char/model.ckpt')
model_path = os.path.join(cur_pos,'./Dataset/models/char/model.ckpt-510')

train_flag = 0 #set flag = 1 to train, other numbers 0 etc. to test
net = char_recognition_cnn_net()

if train_flag == 1:
    # train
    net.train(data_dir,train_model_path)
else:
    # test
    test_X = net.init_testData(test_dir)
    text = net.test(test_X,model_path)
    print(text)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /Users/evan/MS/Ece697/bishe /1/./Dataset/models/char/model.ckpt-510
['zh_su', 'W', 'zh_meng', '8']
